* Project: LSMS_ag_prod 
* Created on: Feb 2025
* Created by: rg
* Edited on: 8 March 2025
* Edited by: rg

* does
	* create summary stats per wave for reach country

* assumes
	* access to aggregated data

* notes:
    * there is no project.do file so file paths have to be updated manually 

In [1]:
# create tables for each country showinf mean and sd per wave
# import libraries 

import pandas as pd
import re

In [2]:
# specify file patht to the data 

path =  '/Users/rodrigoguerra/Library/CloudStorage/OneDrive-UniversityofArizona/weather_and_agriculture/lsms_base/countries/aggregate/allrounds_final_weather_cp.dta'
df = pd.read_stata(path)

# define file path to save tables
save_path = '/Users/rodrigoguerra/Library/CloudStorage/OneDrive-UniversityofArizona/ag_prod/graphs&tables/tables/descriptive_stats/'

/var/folders/wj/dcwc9sc12tz299sg285qbzvr0000gn/T/ipykernel_81841/2006345036.py:4: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(path)
/var/folders/wj/dcwc9sc12tz299sg285qbzvr0000gn/T/ipykernel_81841/2006345036.py:4: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(path)


In [3]:
# list vars in df
print(df.columns.tolist())


['country', 'wave', 'crop_name', 'season', 'pw', 'ea_id_merge', 'ea_id_obs', 'strataid', 'urban', 'admin_1', 'admin_2', 'admin_3', 'lat_modified', 'lon_modified', 'geocoords_id', 'parcel_id_obs', 'parcel_id_merge', 'plot_id_obs', 'plot_id_merge', 'hh_id_obs', 'harvest_end_month', 'planting_month', 'harvest_kg', 'harvest_value_LCU', 'harvest_value_USD', 'seed_kg', 'seed_value_LCU', 'seed_value_USD', 'improved', 'used_pesticides', 'crop_shock', 'pests_shock', 'rain_shock', 'drought_shock', 'flood_shock', 'hh_id_merge', 'plot_area_GPS', 'farm_size', 'yield_kg', 'yield_value_LCU', 'yield_value_USD', 'total_labor_days', 'total_hired_labor_days', 'total_family_labor_days', 'hired_labor_value_LCU', 'hired_labor_value_USD', 'fert_kg', 'inorganic_fertilizer', 'inorganic_fertilizer_value_LCU', 'inorganic_fertilizer_value_USD', 'organic_fertilizer', 'intercropped', 'maincrop_valueshare', 'manager_id_merge', 'manager_id_obs', 'age_manager', 'female_manager', 'formal_education_manager', 'irrigated'

In [4]:
# drop if "crop" is missing and specify variables to be included in the graph
df = df.dropna(subset = ["crop"])

selected_vars =  [
    "yield_cp", "total_labor_days", "fert_value_cp", "plot_area_GPS", "seed_value_cp",
    "used_pesticides",  "organic_fertilizer", "irrigated", "crop_shock", "intercropped",
    "hh_shock", "livestock", "hh_size", "formal_education_manager", "female_manager", 
    "age_manager", "hh_electricity_access", "urban", "plot_owned", "v02_rf1", "v04_rf1", "v09_rf1"
    ]

In [5]:
# check data types of selected variables
categorical_vars = [var for var in selected_vars if df[var].dtype == "object" or df[var].dtype.name == "category"]

# print categorical variables
print("Categorical Variables:", categorical_vars)

# these vars are encoded in stata but read as "categorical" in python



Categorical Variables: ['used_pesticides', 'irrigated', 'crop_shock', 'intercropped', 'livestock', 'formal_education_manager', 'female_manager', 'hh_electricity_access', 'urban', 'plot_owned']


In [6]:
# convert yes/no to 0/1
df[categorical_vars] = df[categorical_vars].apply(lambda x: x.astype(str).str.strip().str.lower().replace({"yes": 1, "no": 0})).astype(float).fillna(0).astype(int)

In [7]:
# dictionary to store tables per country 
country_tables = {}

# dictionary to rename variables (not show default name eg. yield_USD)
rename_dict = {
    "yield_cp": "Yield",
    "total_labor_days": "Total Labor Days",
    "fert_value_cp": "Inorganic Fertilizer", 
    "plot_area_GPS": "Plot Size",
    "seed_value_cp": "Seed",
    "used_pesticides": "Pesticide Use (Yes/No)",
    "organic_fertilizer": "Organic Fertilizer Use (Yes/No)", 
    "irrigated": "Is the Plot Irrigated? (Yes/No)",
    "crop_shock": "Exposure to Agricultural Shock (Yes/No)",
    "intercropped": "Any Crop Intercropped (Yes/No)",
    "hh_shock": "Household Shock (Yes/No)",
    "livestock": "Engaged in Livestock Activities (Yes/No)",
    "hh_size": "Household Size",
    "formal_education_manager": "Any Formal Education (Yes/No)",
    "female_manager": "Is the Plot Manager Female? (Yes/No)",
    "age_manager": "Age",
    "hh_electricity_access": "Electricity Access (Yes/No)",
    "urban": "Urban EA (Yes/No)",
    "plot_owned": "Farmer owns the Plot (Yes/No)",
    "v02_rf1": "Median Dailly Rainfall",
    "v04_rf1": "Skew of Daily Rainfall",
    "v09_rf1": "Deviation in Rainy Days"

}

In [8]:
# define categories for variables
variable_categories = {
    "Yield": "Plot Level",
    "Inorganic Fertilizer": "Plot Level",
    "Seed": "Plot Level",
    "Total Labor Days": "Plot Level",
    "Plot Size": "Plot Level",
    "Pesticide Use (Yes/No)": "Plot Level",
    "Organic Fertilizer Use (Yes/No)": "Plot Level", 
    "Is the Plot Irrigated? (Yes/No)": "Plot Level",
    "Exposure to Agricultural Shock (Yes/No)": "Plot Level",
    "Any Crop Intercropped (Yes/No)": "Plot Level",

    "Household Size": "Household Level",
    "Electricity Access (Yes/No)": "Household Level",
    "Household Shock (Yes/No)": "Household Level",
    "Engaged in Livestock Activities (Yes/No)": "Household Level",
    "Urban EA (Yes/No)": "Household Level",

    "Age": "Farmer Level",
    "Any Formal Education (Yes/No)": "Farmer Level",
    "Is the Plot Manager Female? (Yes/No)": "Farmer Level",
    "Farmer owns the Plot (Yes/No)": "Farmer Level",


    "Median Dailly Rainfall": "Weather",
    "Skew of Daily Rainfall": "Weather",
    "Deviation in Rainy Days": "Weather"
}

In [9]:
# create a loop to generate one table per country
for country in df["country"].unique():
    df_country = df[df["country"] == country]

    # calculate mean and std per wave for selected vars
    summary = df_country.groupby("wave")[selected_vars].agg(['mean', 'std'])

    # transpose to get variables as rows
    summary = summary.T.unstack(level = 1)

    # round to two decimals
    summary = summary.round(2)

    # format column headers 
    new_columns = pd.MultiIndex.from_tuples([
        (f"Wave {wave}", "St. Dev" if stat == "std" else stat.capitalize()) 
        for wave, stat in summary.columns
    ])
    summary.columns = new_columns

    # rename variables
    summary.rename(index = rename_dict, inplace = True)

    # add category level to the index
    summary.index = pd.MultiIndex.from_tuples(
        [(variable_categories[var], var) for var in summary.index],
        names=["Category", "Variable"]
    )

    # set specific order for categories
    category_order = ["Plot Level", "Household Level", "Farmer Level", "Weather"]
    summary.index = summary.index.set_levels(
    pd.CategoricalIndex(summary.index.levels[0], categories = category_order, ordered = True), level = 0
    )
    
    # sort categories so variables within same category are together
    summary = summary.sort_index(level = 0)
    
    # store table in dictionary
    country_tables[country] = summary 

    # print preview
    print(f"\nSummary Table for {country}:")
    print(summary)

    # save as LaTeX with multi-level index
    summary.to_latex(f"{save_path}{country}_summary_stats.tex", 
                     index=True, 
                     caption=f"Summary Statistics for {country}", 
                     label=f"tab:{country}_summary", 
                     multicolumn=True, 
                     multicolumn_format="c", 
                     column_format="ll" + " c c" * (len(summary.columns) // 2),
                     float_format=lambda x: "{:,.2f}".format(x))


Summary Table for Ethiopia:
                                                          Wave 1           \
                                                            Mean  St. Dev   
Category        Variable                                                    
Plot Level      Any Crop Intercropped (Yes/No)              0.26     0.44   
                Exposure to Agricultural Shock (Yes/No)     0.28     0.45   
                Inorganic Fertilizer                        5.23    17.58   
                Is the Plot Irrigated? (Yes/No)             0.02     0.15   
                Organic Fertilizer Use (Yes/No)             0.28     0.45   
                Pesticide Use (Yes/No)                      0.02     0.14   
                Plot Size                                   0.17     0.39   
                Seed                                        5.97    11.72   
                Total Labor Days                           24.15    19.58   
                Yield                          